## Σε αυτό το σημείο της εργασίες πρέπει να συγκρίνουμε τα 3 pipelines που χρησιμοποιήσαμε στο ερώτημα 1Β μεταξύ τους

### Βήματα 
- Διαβάζουμε τα 3 ανακατσκευασμένα κείμενα
- Κάνουμε σύγκριση

In [7]:
original_text=""
pipeline1=""
pipeline2=""
pipeline3=""

In [8]:
try:
    with open('../text_files/text1.txt', 'r', encoding='utf-8') as file:
        original_text = file.read()
    print("File content successfully loaded into original_text.")
except FileNotFoundError:
    print("Error: The file 'text1.txt' was not found. Please make sure the file exists and the path is correct.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

File content successfully loaded into original_text.


In [9]:
try:
    with open("../text_files/first_pipeline_text1.txt", 'r', encoding='utf-8') as file:
        pipeline1 = file.read()
    print("File content successfully loaded into pipeline1.")
except FileNotFoundError:
    print("Error: Please make sure the file exists and the path is correct.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

File content successfully loaded into pipeline1.


In [10]:
try:
    with open("../text_files/second_pipeline_text1.txt", 'r', encoding='utf-8') as file:
        pipeline2 = file.read()
    print("File content successfully loaded into pipeline2.")
except FileNotFoundError:
    print("Error: Please make sure the file exists and the path is correct.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

File content successfully loaded into pipeline2.


In [11]:
try:
    with open("../text_files/third_pipeline_text1.txt", 'r', encoding='utf-8') as file:
        pipeline3 = file.read()
    print("File content successfully loaded into pipeline3.")
except FileNotFoundError:
    print("Error: Please make sure the file exists and the path is correct.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

File content successfully loaded into pipeline3.


## Αρχικά θα συγκρίνουμε το πόσο διαφέρει η κάθε ανακατασκευή από το πρωτότυπο κείμενο με μέθοδο cosine similarity

In [12]:
from collections import Counter
import numpy as np
def cosine_similarity_logic(original_text,pipeline):
    sentences = [original_text, pipeline]
    tokenized = [s.lower().split() for s in sentences]
    vocab = sorted(set(word for doc in tokenized for word in doc))

    tf_matrix = []
    for doc in tokenized:
        word_counts = Counter(doc)
        tf_vector = [word_counts[word] / len(doc) for word in vocab]
        tf_matrix.append(tf_vector)
        
        
    N = len(tokenized) 
    df_vector = [sum(1 for doc in tokenized if word in doc) for word in vocab]
    idf_vector = [np.log((N + 1) / (df + 1)) + 1 for df in df_vector]
    tfidf_matrix = np.array(tf_matrix) * np.array(idf_vector)
    
    
    similarity = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])
    print(f"Cosine Similarity: {similarity}")
    
def cosine_similarity(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [13]:
cosine_similarity_logic(original_text,pipeline1)
cosine_similarity_logic(original_text,pipeline2)
cosine_similarity_logic(original_text,pipeline3)



Cosine Similarity: 0.909638830217755
Cosine Similarity: 0.89091260117103
Cosine Similarity: 0.6972463548228686


## Ground truth κείμενο από το gemini

In [14]:
gemini="""Happy Dragon Boat Festival! I hope you're celebrating safely and enjoying a wonderful time.

Thank you for relaying our message to the doctor regarding his contract review. I actually received the approved message from the professor a couple of days ago. I'm very grateful for the professor's full support with our Springer proceedings publication."""

In [15]:
cosine_similarity_logic(gemini,pipeline1)
cosine_similarity_logic(gemini,pipeline2)
cosine_similarity_logic(gemini,pipeline3)

Cosine Similarity: 0.4912117852064911
Cosine Similarity: 0.5029258632108523
Cosine Similarity: 0.41372297221366444


## Ground truth κείμενο από το deepseek

In [16]:
deepseek="""Today is our Dragon Boat Festival, a cherished celebration in Chinese culture. I wish you safety, joy, and prosperity during this time—may you enjoy the festivities as deeply as I hope for you.

Thank you for sharing the message regarding the doctor’s contract review. I’ve already received the approved notice from the professor a few days ago and sincerely appreciate their unwavering support for our Springer proceedings publication."""

In [17]:
cosine_similarity_logic(deepseek,pipeline1)
cosine_similarity_logic(deepseek,pipeline2)
cosine_similarity_logic(deepseek,pipeline3)

Cosine Similarity: 0.4240647916966377
Cosine Similarity: 0.4173758627748884
Cosine Similarity: 0.3736299567629978


## Ground truth κείμενο από το chat gpt

In [18]:
chat_gpt="""Today is our Dragon Boat Festival, a special celebration in our Chinese culture. It is a time to honor and wish for safety and prosperity in our lives. I hope you also enjoy this festival, as I send you my warmest wishes.

Thank you for your message and for conveying our words to the doctor regarding his upcoming contract review. This is important for all of us.

I received this message to confirm the approval. Actually, the professor shared this with me a couple of days ago. I truly appreciate the professor’s full support for our Springer proceedings publication."""

In [19]:
cosine_similarity_logic(chat_gpt,pipeline1)
cosine_similarity_logic(chat_gpt,pipeline2)
cosine_similarity_logic(chat_gpt,pipeline3)

Cosine Similarity: 0.6111970629422422
Cosine Similarity: 0.6211219678610983
Cosine Similarity: 0.49879739978083254


# Word Embeddings

In [20]:
from nltk.tokenize import word_tokenize

In [21]:
def cosine_similarity(vec1, vec2):
    """
    Υπολογίζει την ομοιότητα συνημιτόνου μεταξύ δύο NumPy διανυσμάτων.
    """
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    
    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0.0 # Αποφυγή διαίρεσης με το μηδέν
    
    return dot_product / (norm_vec1 * norm_vec2)

In [22]:
import torch
import torch.nn as nn

class Word2VecCBOW(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(Word2VecCBOW, self).__init__()
        # Το embedding layer: Μετατρέπει τα integer IDs των λέξεων σε πυκνά διανύσματα.
        # vocab_size: Ο αριθμός των μοναδικών λέξεων στο λεξιλόγιό μας.
        # embedding_dim: Η διάσταση του διανύσματος για κάθε λέξη (π.χ., 100, 300).
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        
        # Το γραμμικό (linear) layer εξόδου:
        # Παίρνει το embedding του πλαισίου και προβλέπει την κεντρική λέξη
        # δίνοντας ένα σκορ για κάθε λέξη στο λεξιλόγιο.
        self.output_layer = nn.Linear(embedding_dim, vocab_size)
        
    def forward(self, inputs):
        # inputs: Ένα tensor με τα IDs των λέξεων του πλαισίου.
        # Αναμενόμενο σχήμα: (batch_size, context_window_size)
        
        # Βήμα 1: Λάβετε τα embeddings για κάθε λέξη του πλαισίου.
        # Το self.embeddings(inputs) θα επιστρέψει ένα tensor σχήματος:
        # (batch_size, context_window_size, embedding_dim)
        embeds = self.embeddings(inputs)
        
        # Βήμα 2: Υπολογίστε το embedding του πλαισίου.
        # Για το CBOW, αυτό είναι συνήθως ο μέσος όρος των embeddings των λέξεων του πλαισίου.
        # Το dim=1 σημαίνει ότι υπολογίζουμε το μέσο όρο κατά μήκος της διάστασης του "πλαισίου".
        # Αποτέλεσμα: (batch_size, embedding_dim)
        context_embed = torch.mean(embeds, dim=1)
        
        # Βήμα 3: Περάστε το embedding του πλαισίου από το output layer.
        # Αυτό παράγει τα "logits" ή "σκορ" για κάθε λέξη στο λεξιλόγιο.
        # Αποτέλεσμα: (batch_size, vocab_size)
        output = self.output_layer(context_embed)
        
        return output

In [23]:
def get_paragraph_embedding(paragraph, model, word_to_idx):
    """
    Δημιουργεί ένα ενιαίο διάνυσμα (embedding) για μια παράγραφο
    κάνοντας τον μέσο όρο των embeddings των λέξεων που την απαρτίζουν.
    Χρησιμοποιεί το embedding layer από το δικό σας μοντέλο.
    """
    tokens = [word.lower() for word in word_tokenize(paragraph)]
    
    # Συλλέξτε τα IDs των λέξεων που υπάρχουν στο λεξιλόγιο
    word_ids = []
    for word in tokens:
        if word in word_to_idx:
            word_ids.append(word_to_idx[word])
    
    if not word_ids:
        # Αν δεν βρέθηκαν λέξεις στο λεξιλόγιο, επιστρέψτε ένα διάνυσμα μηδενικών
        # με τη σωστή διάσταση του embedding
        return np.zeros(model.embeddings.embedding_dim)
    
    # Μετατροπή των IDs σε PyTorch tensor
    # Προσθήκη μιας διάστασης batch (unsqueeze(0)) γιατί το embedding layer περιμένει batch
    input_tensor = torch.tensor(word_ids, dtype=torch.long).unsqueeze(0) # (1, num_words)
    
    # Λάβετε τα embeddings από το embedding layer του μοντέλου
    # Δεν χρειάζεται να περάσετε από ολόκληρο το forward pass του μοντέλου
    # γιατί θέλουμε μόνο τα embeddings, όχι τις προβλέψεις του output layer.
    with torch.no_grad(): # Δεν χρειαζόμαστε gradients για απλή ανάκτηση embeddings
        word_embeds = model.embeddings(input_tensor) # (1, num_words, embedding_dim)
    
    # Υπολογίστε τον μέσο όρο των embeddings για να πάρετε το embedding της παραγράφου
    paragraph_embedding = torch.mean(word_embeds, dim=1).squeeze(0) # (embedding_dim,)
    
    return paragraph_embedding.numpy() # Επιστροφή ως NumPy array για υπολογισμό ομοιότητας

In [24]:
# --- 2. Προετοιμασία Δεδομένων και Λεξιλογίου ---

from collections import defaultdict

all_paragraphs = [pipeline1, pipeline2, pipeline3]
all_words = []
for p in all_paragraphs:
    all_words.extend([word.lower() for word in word_tokenize(p)]) # word_tokenize τώρα είναι defined

word_to_idx = defaultdict(lambda: len(word_to_idx))
idx_to_word = {}

for word in all_words:
    if word not in word_to_idx:
        idx = word_to_idx[word]
        idx_to_word[idx] = word

VOCAB_SIZE = len(word_to_idx)
EMBEDDING_DIM = 100

print(f"Δημιουργήθηκε λεξιλόγιο με {VOCAB_SIZE} λέξεις.")
print("-" * 50)

# --- 3. Αρχικοποίηση του Μοντέλου ---
model = Word2VecCBOW(VOCAB_SIZE, EMBEDDING_DIM)
print("Το μοντέλο Word2VecCBOW αρχικοποιήθηκε.")
print(f"Διαστάσεις embedding layer: {model.embeddings.weight.shape}")
print("-" * 50)

Δημιουργήθηκε λεξιλόγιο με 137 λέξεις.
--------------------------------------------------
Το μοντέλο Word2VecCBOW αρχικοποιήθηκε.
Διαστάσεις embedding layer: torch.Size([137, 100])
--------------------------------------------------


In [25]:
print("\n--- Συγκρίσεις Παραγράφων με τη Δική σας Κλάση ---")

embedding1 = get_paragraph_embedding(pipeline1, model, word_to_idx)
embedding2 = get_paragraph_embedding(pipeline2, model, word_to_idx)
embedding3 = get_paragraph_embedding(pipeline3, model, word_to_idx)

print(f"Παράγραφος 1: \"{pipeline1}\"")
print(f"Παράγραφος 2: \"{pipeline2}\"")
print(f"Παράγραφος 3: \"{pipeline3}\"")
print("-" * 50)

similarity_1_2 = cosine_similarity(embedding1, embedding2)
print(f"Ομοιότητα μεταξύ Παραγράφου 1 και Παραγράφου 2: {similarity_1_2:.4f}")

similarity_1_3 = cosine_similarity(embedding1, embedding3)
print(f"Ομοιότητα μεταξύ Παραγράφου 1 και Παραγράφου 3: {similarity_1_3:.4f}")

similarity_1_4 = cosine_similarity(embedding2, embedding3)
print(f"Ομοιότητα μεταξύ Παραγράφου 1 και Παραγράφου 4: {similarity_1_4:.4f}")

print("\nΣημείωση: Τα αποτελέσματα ομοιότητας εδώ είναι πιθανόν τυχαία ή πολύ αδύναμα,")
print("καθώς το μοντέλο Word2VecCBOW δεν έχει εκπαιδευτεί σε ουσιαστικά δεδομένα.")
print("Για να δείτε πραγματικά σημασιολογικά αποτελέσματα, το μοντέλο χρειάζεται εκτενή εκπαίδευση.")


--- Συγκρίσεις Παραγράφων με τη Δική σας Κλάση ---
Παράγραφος 1: "Today is our dragon boat festival, in our Chinese culture, to celebrate it with all safety and great in our lives.Hope you too, enjoy it as my deepest wishes.Thank you for your message to show our words to the doctor, as his next contract checking, to all of us.I got this message to see the approved message.In fact, I have received the message from the professor, to show me, this, a couple of days ago.I am very appreciated the full support of the professor, for our Springer proceedings publication."
Παράγραφος 2: "Today is our dragon boat festival, in our Chinese culture, to celebrate it with all safety and great in our lives.Hope you too, to enjoy it as my deepest wishes.Thank you for your message to show our words to the doctor, as his next contract checks, to all of us.I got this message to see the approved message.In fact, I have received the message from the professor, to show me, this, a couple of days ago.I am ve